In [7]:
"""For use with https://www.cutlistoptimizer.com/"""

# Input Data
rec_dims_in = [
"2.5x2.5",
"1x1",
"3x3",
"2x2.5",
"3x8",
"2.5x3.5",
"3x3.5",
"1x2",
"2.5x3.5",
"3x3.5",
"3.75x2",
"3.75x2",
"4x4",
"7.5x3/16",
"2.25x3.25",
"2.75x4.8",
"1.5x4.17",
"2.25x3.25",
"1.5x0.25",
"7.5x1",
"2x2",
"3.25x1",
"3.75x2",
"1.25x1.25",
"3x3.5"
        ]
bin_dims_in = ["10.25x14.25",
               "10.25x14.25",]

In [8]:
from enum import Enum
from typing import Union

import coolname
from pydantic import BaseModel, Field, validator
class UnitOfMeasure(str, Enum):
    inch = "in"
    millimeter = "mm"

def parse_fraction(v: Union[str, float]) -> float:
    if isinstance(v,str) and "/" in v:
        numerator, denominator = [float(val) for val in v.split("/")]
        v = numerator/denominator
    return float(v)
class Rectangle(BaseModel):
    Length: float
    @validator("Length", pre=True)
    def Length_valid(cls, v: Union[str, float]) -> float:
            return parse_fraction(v)

    Width: float
    @validator("Width", pre=True)
    def Width_valid(cls, v: Union[str, float]) -> float:
        return parse_fraction(v)

    Label: str = Field(default_factory=coolname.generate_slug)
    unit_of_measure: UnitOfMeasure


In [9]:
def convert_hxw_str_rec_to_rec_model(rec: str):
    h, w = rec.split("x")
    return Rectangle(Length=h, Width=w, unit_of_measure=UnitOfMeasure.inch)

def convert_rec_in_to_mm(rec: Rectangle):
    if rec.unit_of_measure != UnitOfMeasure.inch:
        raise RuntimeError("Input rectangle is not in inches!\n"
                           f"Actual unit of measure: {rec.unit_of_measure}")
    in_to_mm_conversion_factor = 25.4
    return Rectangle(Length=(rec.Length * in_to_mm_conversion_factor),
                     Width=(rec.Width * in_to_mm_conversion_factor),
                     Label=rec.Label,
                     unit_of_measure=UnitOfMeasure.millimeter)
inserts_in = [convert_hxw_str_rec_to_rec_model(rec) for rec in rec_dims_in]
inserts_mm = [convert_rec_in_to_mm(insert) for insert in inserts_in]

bin_dims_in = [convert_hxw_str_rec_to_rec_model(rec) for rec in bin_dims_in]
bin_dims_mm = [convert_rec_in_to_mm(rec) for rec in bin_dims_in]


In [10]:
def add_mm_spacing_to_rec(rec: Rectangle, spacing: float):
    return Rectangle(Length=(rec.Length + spacing),
                         Width=(rec.Width + spacing),
                         Label=rec.Label,
                         unit_of_measure=UnitOfMeasure.millimeter)

def add_pocket_mm_spacing_to_rec(rec: Rectangle):
    insert_pocket_space_per_side = 3
    insert_pocket_space_per_dim = insert_pocket_space_per_side*2
    return add_mm_spacing_to_rec(rec, insert_pocket_space_per_dim)

def add_interpocket_mm_spacing_to_rec(rec: Rectangle):
    interpocket_space_per_side = 5
    interpocket_space_per_dim = interpocket_space_per_side*2
    return add_mm_spacing_to_rec(rec, interpocket_space_per_dim)

inserts_mm_spaced = [add_interpocket_mm_spacing_to_rec(add_pocket_mm_spacing_to_rec(insert)) for insert in inserts_mm]
bin_dims_mm_spaced = [add_interpocket_mm_spacing_to_rec(rec) for rec in bin_dims_mm]

In [28]:
import pandas as pd
cut_list_inserts_dfs = pd.concat([pd.Series(insert.dict()) for insert in inserts_mm_spaced], axis=1).T
cut_list_bins_dfs = pd.concat([pd.Series(rec.dict()) for rec in bin_dims_mm_spaced], axis=1).T

In [29]:
for df in [cut_list_inserts_dfs, cut_list_bins_dfs]:
    df.drop("unit_of_measure", axis=1, inplace=True)
    df["Qty"] = 1
    df["Enabled"] = True

In [ ]:
cut_list_inserts_dfs

In [31]:
cut_list_inserts_dfs.to_csv("recs.csv", index=False)
cut_list_bins_dfs.to_csv("bins.csv", index=False)